# Homework 08
This homework is based on the clustering lectures. Check the lecture notes and TA notes - they should help!

In [1]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.1     ✔ stringr   1.5.2
✔ ggplot2   4.0.0     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.1.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


## Question 1
This question will walk you through creating your own `kmeans` function.

#### a) What are the steps of `kmeans`?
**Hint**: There are 4 steps/builder functions that you'll need.

1. Assign each point to a cluster N at random.  
2. Calculate the mean position of each cluster using the previous assignments.  
3. Loop through the points - assign each point to the cluster to whose center it is closest.  
4. Repeat this process until the centers stop moving around.

#### b) Create the builder function for step 1.

In [2]:
builder <- function(n_points, n_clusters){
  sample(((1:n_points) %% n_clusters)+1, n_points, replace=F)
}

#### c) Create the builder function for step 2.

In [3]:
get_cluster_means <- function(data, labels){
  data %>%
    mutate(label__ = labels) %>%
    group_by(label__) %>%
    summarize(across(everything(), mean), .groups = "drop") %>%
    arrange(label__)
}

#### d) Create the builder function for step 3.
*Hint*: There are two ways to do this part - one is significantly more efficient than the other. You can do either.  

In [4]:
assign_cluster_fast <- function(data, means){
  data_matrix <- as.matrix(data)
  means_matrix <- as.matrix(means %>% dplyr::select(-label__))
  dii <- sort(rep(1:nrow(data), nrow(means)))
  mii <- rep(1:nrow(means), nrow(data))
  data_repped <- data_matrix[dii, ]
  means_repped <- means_matrix[mii, ]
  diff_squared <- (data_repped - means_repped)^2
  all_distances <- rowSums(diff_squared)
  tibble(dii=dii, mii=mii, distance=all_distances) %>%
    group_by(dii) %>%
    arrange(distance) %>%
    filter(row_number()==1) %>%
    ungroup() %>%
    arrange(dii) %>%
    pull(mii)
}

assign_cluster_slow <- function(data, means){
  dii <- 1:nrow(data)
  cii <- 1:nrow(means)
  labels <- c()
  for(point_index in dii){
    smallest_dist <- Inf
    smallest_label <- NA
    for(clus_index in cii){
      point <- data[point_index, ]
      clus <- means %>% dplyr::select(-label__) %>% `[`(clus_index, )
      diff <- point - clus
      dist <- sum(diff * diff)
      if(dist < smallest_dist){
        smallest_dist <- dist
        smallest_label <- means[clus_index, ]$label__
      }
    }
    labels <- c(labels, smallest_label)
  }
  labels
}

#### e) Create the builder function for step 4.

In [5]:
kmeans_done <- function(old_means, new_means, eps=1e-6){
  om <- as.matrix(old_means)
  nm <- as.matrix(new_means)
  m <- mean(sqrt(rowSums((om - nm)^2)))
  if(m < eps) TRUE else FALSE
}

mykmeans <- function(data, n_clusters, eps=1e-6, max_it = 1000, verbose = FALSE){
  labels <- builder(nrow(data), n_clusters)
  old_means <- get_cluster_means(data, labels)
  done <- FALSE
  it <- 0
  while(!done & it < max_it){
    labels <- assign_cluster_fast(data, old_means)
    new_means <- get_cluster_means(data, labels)
    if(kmeans_done(old_means, new_means)){
      done <- TRUE
    } else {
      old_means <- new_means
      it <- it + 1
      if(verbose){
        cat(sprintf("%d\n", it))
      }
    }
  }
  list(labels=labels, means=new_means)
}

#### f) Combine them all into your own `kmeans` function.

In [9]:
label_randomly <- function(n_points, n_clusters){
  sample(((1:n_points) %% n_clusters)+1, n_points, replace=F)
}

get_cluster_means <- function(data, labels){
  data %>%
    mutate(label__ = labels) %>%
    group_by(label__) %>%
    summarize(across(everything(), mean), .groups = "drop") %>%
    arrange(label__)
}

assign_cluster_fast <- function(data, means){
  data_matrix <- as.matrix(data)
  means_matrix <- as.matrix(means %>% dplyr::select(-label__))
  dii <- sort(rep(1:nrow(data), nrow(means)))
  mii <- rep(1:nrow(means), nrow(data))
  data_repped <- data_matrix[dii, ]
  means_repped <- means_matrix[mii, ]
  diff_squared <- (data_repped - means_repped)^2
  all_distances <- rowSums(diff_squared)
  tibble(dii=dii, mii=mii, distance=all_distances) %>%
    group_by(dii) %>%
    arrange(distance) %>%
    filter(row_number()==1) %>%
    ungroup() %>%
    arrange(dii) %>%
    pull(mii)
}

assign_cluster_slow <- function(data, means){
  dii <- 1:nrow(data)
  cii <- 1:nrow(means)
  labels <- c()
  for(point_index in dii){
    smallest_dist <- Inf
    smallest_label <- NA
    for(clus_index in cii){
      point <- data[point_index, ]
      clus <- means %>% dplyr::select(-label__) %>% `[`(clus_index, )
      diff <- point - clus
      dist <- sum(diff * diff)
      if(dist < smallest_dist){
        smallest_dist <- dist
        smallest_label <- means[clus_index, ]$label__
      }
    }
    labels <- c(labels, smallest_label)
  }
  labels
}

kmeans_done <- function(old_means, new_means, eps=1e-6){
  om <- as.matrix(old_means)
  nm <- as.matrix(new_means)
  m <- mean(sqrt(rowSums((om - nm)^2)))
  if(m < eps) TRUE else FALSE
}

mykmeans <- function(data, n_clusters, eps=1e-6, max_it = 1000, verbose = FALSE){
  labels <- label_randomly(nrow(data), n_clusters)
  old_means <- get_cluster_means(data, labels)
  done <- FALSE
  it <- 0
  while(!done & it < max_it){
    labels <- assign_cluster_fast(data, old_means)
    new_means <- get_cluster_means(data, labels)
    if(kmeans_done(old_means, new_means)){
      done <- TRUE
    } else {
      old_means <- new_means
      it <- it + 1
      if(verbose){
        cat(sprintf("%d\n", it))
      }
    }
  }
  list(labels=labels, means=new_means)
}

## Question 2
This is when we'll test your `kmeans` function.
#### a) Read in the `voltages_df.csv` data set.

In [6]:
vdf <- read_csv("voltages_df.csv")

Rows: 900 Columns: 250
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (250): 0, 1.00401606425703, 2.00803212851406, 3.01204819277108, 4.016064...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


#### b) Call your `kmeans` function with 3 clusters. Print the results with `results$labels` and `results$means`.

In [34]:
myresults <- mykmeans(vdf, 3)

print(myresults$labels)
print(myresults$means)

  [1] 2 3 3 3 3 3 1 1 3 2 2 2 1 2 3 2 2 2 2 2 3 3 3 3 3 2 2 2 1 1 1 1 3 1 1 3 3
 [38] 2 1 3 2 2 2 1 3 1 1 3 1 1 3 1 2 2 1 2 3 2 1 3 2 2 3 2 1 1 3 1 2 1 2 3 1 1
 [75] 3 3 2 1 1 1 1 1 3 1 1 3 3 3 2 1 1 1 3 1 3 3 1 3 3 1 1 1 3 1 2 2 1 2 2 3 2
[112] 2 3 1 2 1 2 2 3 3 3 3 2 2 1 3 3 1 2 1 3 2 3 1 2 3 3 3 2 1 2 3 3 1 2 2 3 2
[149] 2 1 2 1 3 3 1 2 1 1 3 2 1 1 2 3 2 1 3 2 3 2 2 1 3 2 1 1 3 1 2 3 1 3 1 2 1
[186] 3 3 2 3 2 2 3 2 2 1 1 2 3 2 2 1 2 1 1 3 2 3 1 3 2 3 1 2 1 1 3 1 1 3 3 3 3
[223] 3 3 3 1 1 1 1 2 3 2 1 3 3 1 1 2 2 1 1 2 1 1 3 1 2 1 1 3 1 3 2 1 3 1 1 2 2
[260] 2 3 3 1 2 3 3 3 1 1 1 2 3 3 3 1 2 1 1 3 3 1 2 1 2 2 3 1 3 3 1 1 2 1 1 2 3
[297] 2 1 1 3 3 1 2 2 2 1 2 3 3 3 2 2 2 2 2 2 3 2 3 2 2 1 3 1 2 2 2 2 3 2 3 2 1
[334] 2 3 3 1 1 2 3 1 3 1 1 2 1 3 1 2 2 1 3 3 3 2 1 3 2 1 1 3 3 3 3 2 1 2 3 2 2
[371] 2 2 2 3 3 1 1 2 2 2 2 1 1 2 2 1 1 3 3 1 2 3 3 3 3 1 3 3 1 2 2 2 2 3 3 2 2
[408] 1 1 1 3 1 1 2 3 3 3 1 3 3 3 2 2 3 2 1 3 3 3 3 1 1 3 1 2 3 2 2 2 1 1 1 1 2
[445] 3 2 2 2 1 3 2 2 2 3 3 2 3 3 2 1 3 

#### c) Call R's `kmeans` function with 3 clusters. Print the results with `results$labels` and `results$cluster`.
*Hint*: Use the `as.matrix()` function to make the `voltages_df` data frame a matrix before calling `kmeans()`.

In [43]:
vdf_matrix <- as.matrix(vdf)
results <- kmeans(vdf_matrix, centers = 3)

print(results$centers)
print(results$cluster)

          0 1.00401606425703 2.00803212851406 3.01204819277108 4.01606425702811
1 -1.031463        0.9381238        0.7619864        0.3631543       -1.1179412
2 -1.031463        1.2439759        1.0924697        0.9004440        0.3011754
3 -1.031463        1.3093239        1.1616772        0.9787498        0.6481497
  5.02008032128514 6.02409638554217 7.0281124497992 8.03212851405623
1        -1.051145       -0.9766807      -0.8694758       -0.6892375
2        -1.159714       -1.1098127      -1.0685484       -1.0338649
3        -1.168610       -1.1196122      -1.0590962       -0.9943176
  9.03614457831325 10.0401606425703 11.0441767068273 12.0481927710843
1       -0.5661321       -0.2497152        0.6027358        0.6960355
2       -1.0022396       -0.9699741       -0.9343697       -0.8943605
3       -0.9237437       -0.8457536       -0.7572129       -0.6201221
  13.0522088353414 14.0562248995984 15.0602409638554 16.0642570281125
1        0.3917273       -0.9473202       -1.0883844  

#### d) Are your labels/clusters the same? If not, why? Are your means the same?

The labels are not the same due to randomization and no set seed. All means for both R's kmeans and my own is -1.03.

## Question 3
#### a) Explain the process of using a for loop to assign clusters for kmeans.

A for loop calculates the distance from every cluster center for each data point and then assigns the closest one to it.

#### b) Explain the process of vectorizing the code to assign clusters for kmeans.

Instead of running a calculation for each individual data point, vectorizing the code creating arrays that are able to run all distances at once to determine assignments to clusters.

#### c) State which (for loops or vectorizing) is more efficient and why.

While a slower process to run, vectorizing has the benefit of speed of finding clusters once completed, reaching better performance than for loops when using large datasets, and producing a cleaner line of code compared to the bulkiness of nesting for loops inside each other.

## Question 4
#### When does `kmeans` fail? What assumption does `kmeans` use that causes it to fail in this situation?

'kmeans' has the potential to fail when one or many of the assumptions are violated. Those assumptions are the following:
1. The data is in a vector space *(Fails with categorical data or non-linear manifolds)*
2. The clusters are characterized by their centers or centroids *(Fails when clusters are elongated or concaving shapes)*
3. Cluster membership falls off at a similar rate from all centroids *(Fails when varience is unequal between clusters)*
4. Clusters are about the same size *(Fails when clusters are differing sizes or densities)*  
  
Together, kmeans fails when the clusters are not uniform in size and spherical-shape

## Question 5
#### What assumption do Guassian mixture models make?

Gussian mixture models assume that compared to kmeans, there is a significantly less chance of failure if clusters vary in size and shape by drawing data from N Gaussian distributions.

## Question 6
#### What assumption does spectral clustering make? Why does this help us?

The closer two points are to one another, the more likely they are to belong to the same cluster. This helps us because this assumption is not restricted by space, size, or any other componet that kmeans could otherwise fail from, it is a simple guide when dealing with complex distributions of data.

## Question 7
#### Define the gap statistic method. What do we use it for?

The basic idea is to compare the clustering for each value of K to a cluster of data "randomized" into the same domain as the original data. We then compute the dispersion of the two clusterings and look at the difference. We look for a "knee" and that is our cluster number. This is ad-hoc but at least standard.

The gap statistic method determines meanfulness of clusters by comparing our data against randomized data and viewing the difference in result. We use this when determining the number of clusters to assign our data to.